In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [4]:
yeast_strict_real = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen_real = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be_real = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [10]:
for seed in random_seeds:
    real_tr_te_name = 'Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(yeast_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)

In [5]:
yeast_w2v_model_strict = pred.make_w2v_model(yeast_strict_real, 'Yeast/models/yeast_strict')
yeast_w2v_model_gen = pred.make_w2v_model(yeast_gen_real, 'Yeast/models/yeast_gen')
yeast_w2v_model_be = pred.make_w2v_model(yeast_be_real, 'Yeast/models/yeast_be')

2017-05-16 14:30:14,727 : INFO : collecting all words and their counts
2017-05-16 14:30:14,728 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-16 14:30:14,765 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-05-16 14:30:14,765 : INFO : Loading a fresh vocabulary
2017-05-16 14:30:14,780 : INFO : min_count=6 retains 2978 unique words (26% of original 11431, drops 8453)
2017-05-16 14:30:14,781 : INFO : min_count=6 leaves 173585 word corpus (92% of original 188505, drops 14920)
2017-05-16 14:30:14,791 : INFO : deleting the raw counts dictionary of 11431 items
2017-05-16 14:30:14,793 : INFO : sample=0.0001 downsamples 549 most-common words
2017-05-16 14:30:14,794 : INFO : downsampling leaves estimated 74593 word corpus (43.0% of prior 173585)
2017-05-16 14:30:14,795 : INFO : estimated required memory for 2978 words and 600 dimensions: 15783400 bytes
2017-05-16 14:30:14,802 : INFO : resetting layer weights
2017-0

Parsing datasets sentences
Training Word2Vec Model


2017-05-16 14:30:15,589 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-16 14:30:15,600 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-16 14:30:15,602 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-16 14:30:15,605 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-16 14:30:15,606 : INFO : training on 942525 raw words (373011 effective words) took 0.7s, 498521 effective words/s
2017-05-16 14:30:15,606 : INFO : precomputing L2-norms of word weight vectors
2017-05-16 14:30:15,624 : INFO : saving Word2Vec object under ../../Results/Yeast/models/yeast_strict_model, separately None
2017-05-16 14:30:15,625 : INFO : not storing attribute syn0norm
2017-05-16 14:30:15,625 : INFO : not storing attribute cum_table
2017-05-16 14:30:16,142 : INFO : saved ../../Results/Yeast/models/yeast_strict_model


Parsing datasets sentences


2017-05-16 14:30:16,668 : INFO : collecting all words and their counts
2017-05-16 14:30:16,669 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-16 14:30:16,709 : INFO : PROGRESS: at sentence #10000, processed 240608 words, keeping 11144 word types
2017-05-16 14:30:16,752 : INFO : PROGRESS: at sentence #20000, processed 483529 words, keeping 14874 word types
2017-05-16 14:30:16,795 : INFO : PROGRESS: at sentence #30000, processed 727180 words, keeping 17441 word types
2017-05-16 14:30:16,838 : INFO : PROGRESS: at sentence #40000, processed 969859 words, keeping 19663 word types


...

2017-05-16 14:30:16,871 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-05-16 14:30:16,872 : INFO : Loading a fresh vocabulary
2017-05-16 14:30:16,893 : INFO : min_count=6 retains 7355 unique words (34% of original 21147, drops 13792)
2017-05-16 14:30:16,894 : INFO : min_count=6 leaves 1130505 word corpus (97% of original 1155938, drops 25433)
2017-05-16 14:30:16,913 : INFO : deleting the raw counts dictionary of 21147 items
2017-05-16 14:30:16,914 : INFO : sample=0.0001 downsamples 521 most-common words
2017-05-16 14:30:16,915 : INFO : downsampling leaves estimated 513926 word corpus (45.5% of prior 1130505)
2017-05-16 14:30:16,916 : INFO : estimated required memory for 7355 words and 600 dimensions: 38981500 bytes
2017-05-16 14:30:16,945 : INFO : resetting layer weights
2017-05-16 14:30:17,058 : INFO : training model with 4 workers on 7355 vocabulary and 600 features, using sg=0 hs=0 sample=0.0001 negative=5 window=7
2017-05-16 14:30:17

Parsing datasets sentences


2017-05-16 14:30:25,563 : INFO : collecting all words and their counts
2017-05-16 14:30:25,563 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-16 14:30:25,604 : INFO : PROGRESS: at sentence #10000, processed 229990 words, keeping 12269 word types
2017-05-16 14:30:25,642 : INFO : PROGRESS: at sentence #20000, processed 459245 words, keeping 16803 word types
2017-05-16 14:30:25,685 : INFO : PROGRESS: at sentence #30000, processed 688639 words, keeping 20010 word types
2017-05-16 14:30:25,728 : INFO : PROGRESS: at sentence #40000, processed 916935 words, keeping 22755 word types


Training Word2Vec Model


2017-05-16 14:30:25,771 : INFO : PROGRESS: at sentence #50000, processed 1144943 words, keeping 25062 word types
2017-05-16 14:30:25,818 : INFO : PROGRESS: at sentence #60000, processed 1373907 words, keeping 27120 word types
2017-05-16 14:30:25,861 : INFO : PROGRESS: at sentence #70000, processed 1602580 words, keeping 29006 word types
2017-05-16 14:30:25,906 : INFO : PROGRESS: at sentence #80000, processed 1832124 words, keeping 30626 word types
2017-05-16 14:30:25,950 : INFO : PROGRESS: at sentence #90000, processed 2061054 words, keeping 32180 word types
2017-05-16 14:30:25,993 : INFO : PROGRESS: at sentence #100000, processed 2288480 words, keeping 33620 word types
2017-05-16 14:30:26,042 : INFO : PROGRESS: at sentence #110000, processed 2516418 words, keeping 34947 word types
2017-05-16 14:30:26,090 : INFO : PROGRESS: at sentence #120000, processed 2746160 words, keeping 36208 word types
2017-05-16 14:30:26,137 : INFO : PROGRESS: at sentence #130000, processed 2975457 words, keep

In [7]:
for seed in random_seeds:
    data_name = '../../Results/Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
    train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
    test_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
    test_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_strict, feature_count=600)

    strict_list_SR_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_gen, feature_count=600)

    strict_list_GEN_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                  train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_be, feature_count=600)

    strict_list_BE_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    pickle.dump(strict_list_SR_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_GEN_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_BE_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'wb'))

In [5]:
def modelfit(alg, train_vecs, train_labels, w2v_model_type, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_vecs, 
                              label=train_labels)
        cvresult = xgb.cv(xgb_param, 
                          xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds, 
                          metrics='auc', 
                          early_stopping_rounds=50)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    #fit the algorithm on the data
    alg.fit(train_vecs, train_labels, eval_metric='auc')
    
    #Predict training set:
    train_predictions = alg.predict(train_vecs)
    train_predprob = alg.predict_proba(train_vecs)[:,1]
    
    #Print Model report:
    print(w2v_model_type, '\nModel Report')
    print(w2v_model_type, 'Accuracy: %.4g' % metrics.accuracy_score(train_labels, train_predictions))
    print(w2v_model_type, 'AUC Score (Train): %f' % metrics.roc_auc_score(train_labels, train_predprob))
    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importance')
#     plt.ylabel('Feature Importance Score')

    error = 1-metrics.accuracy_score(train_labels, train_predictions)
    auc = metrics.roc_auc_score(train_labels, train_predprob)
    
    return error, auc

In [6]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.6566
STRICT AUC Score (Train): 0.740780
GEN 
Model Report
GEN Accuracy: 0.6362
GEN AUC Score (Train): 0.714120
BE 
Model Report
BE Accuracy: 0.6472
BE AUC Score (Train): 0.736452
235
STRICT 
Model Report
STRICT Accuracy: 0.6621
STRICT AUC Score (Train): 0.761532
GEN 
Model Report
GEN Accuracy: 0.6273
GEN AUC Score (Train): 0.695405
BE 
Model Report
BE Accuracy: 0.65
BE AUC Score (Train): 0.749577
905
STRICT 
Model Report
STRICT Accuracy: 0.6447
STRICT AUC Score (Train): 0.742371
GEN 
Model Report
GEN Accuracy: 0.6378
GEN AUC Score (Train): 0.709266
BE 
Model Report
BE Accuracy: 0.6521
BE AUC Score (Train): 0.738487
2895
STRICT 
Model Report
STRICT Accuracy: 0.6724
STRICT AUC Score (Train): 0.771002
GEN 
Model Report
GEN Accuracy: 0.6532
GEN AUC Score (Train): 0.726471
BE 
Model Report
BE Accuracy: 0.6596
BE AUC Score (Train): 0.754778
3462
STRICT 
Model Report
STRICT Accuracy: 0.6795
STRICT AUC Score (Train): 0.780634
GEN 
Model Report
GEN Ac

In [7]:
print('Strict Results Error: ', np.mean(strict_error_list), ', Auc: ', np.mean(strict_auc_list))
print('Gen Results Error: ', np.mean(gen_error_list), ', Auc: ', np.mean(gen_auc_list))
print('Be Results Error: ', np.mean(be_error_list), ', Auc: ', np.mean(be_auc_list))

Strict Results Error:  0.330430479588 , Auc:  0.768074209551
Gen Results Error:  0.364579946523 , Auc:  0.707197019221
Be Results Error:  0.344158176436 , Auc:  0.74322406605


In [8]:
param_test1 = {
    'max_depth':[3,4,5,6,7,8,9,10],
    'min_child_weight':[1,2,3,4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=140, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8, 
                                                  objective='binary:logistic', 
                                                  nthread=4, 
                                                  scale_pos_weight=1, 
                                                  seed=24), 
                        param_grid=param_test1, 
                        scoring='roc_auc',  
                        iid=False, 
                        cv=5)

In [10]:
strict_param_test1 = []
for seed in [144]:
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(strict_list[0], strict_list[2])
    strict_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [11]:
gen_param_test1 = []
for seed in [144]:
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(gen_list[0], gen_list[2])
    gen_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [12]:
be_param_test1 = []
for seed in [144]:
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(be_list[0], be_list[2])
    be_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [13]:
print(strict_param_test1, gen_param_test1, be_param_test1)

[(144, ({'max_depth': 3, 'min_child_weight': 1}, 0.5184627231701338))] [(144, ({'max_depth': 3, 'min_child_weight': 5}, 0.5033950980591407))] [(144, ({'max_depth': 6, 'min_child_weight': 6}, 0.4984421837963776))]


In [14]:
xgb2 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=3, 
                     min_child_weight=3, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb2, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb2, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb2, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.712
STRICT AUC Score (Train): 0.869563
GEN 
Model Report
GEN Accuracy: 0.6677
GEN AUC Score (Train): 0.787012
BE 
Model Report
BE Accuracy: 0.5891
BE AUC Score (Train): 0.640011
235
STRICT 
Model Report
STRICT Accuracy: 0.581
STRICT AUC Score (Train): 0.660918
GEN 
Model Report
GEN Accuracy: 0.586
GEN AUC Score (Train): 0.627057
BE 
Model Report
BE Accuracy: 0.5887
BE AUC Score (Train): 0.651093
905
STRICT 
Model Report
STRICT Accuracy: 0.5811
STRICT AUC Score (Train): 0.656544
GEN 
Model Report
GEN Accuracy: 0.5906
GEN AUC Score (Train): 0.629023
BE 
Model Report
BE Accuracy: 0.5933
BE AUC Score (Train): 0.649648
2895
STRICT 
Model Report
STRICT Accuracy: 0.5872
STRICT AUC Score (Train): 0.674281
GEN 
Model Report
GEN Accuracy: 0.5776
GEN AUC Score (Train): 0.640244
BE 
Model Report
BE Accuracy: 0.5947
BE AUC Score (Train): 0.655433
3462
STRICT 
Model Report
STRICT Accuracy: 0.5994
STRICT AUC Score (Train): 0.673769
GEN 
Model Report
GEN Acc